In [36]:
import os
import torch
import fairseq
from fairseq.modules import GumbelVectorQuantizer
if 'cd' not in globals():
    os.chdir('..')
    cd = True


In [37]:
wav2vec, wav2vec_cfg = fairseq.checkpoint_utils.load_model_ensemble(['wav2vec/wav2vec_small.pt'])

In [8]:
model =  wav2vec[0]

In [13]:
model.quantizer.num_vars

320

In [19]:
model.quantizer.groups

2

In [16]:
model.quantizer.vars.size()

torch.Size([1, 640, 128])

In [52]:
audio = torch.randn(2, 10000)

In [53]:
f = model.feature_extractor(audio).transpose(1, 2)

In [34]:
q = model.quantizer(f)
q

{'num_vars': 640,
 'code_perplexity': tensor(4.9826),
 'prob_perplexity': tensor(369.7832, grad_fn=<SumBackward0>),
 'temp': 2.0,
 'x': tensor([[[ 0.0523, -0.0557,  0.0898,  ..., -0.0665,  0.0225,  0.0593],
          [-0.1458,  0.0730,  0.3328,  ...,  0.1575,  0.3792,  0.0783],
          [ 0.1437,  0.1912,  0.4458,  ...,  0.1274,  0.0856,  0.2498],
          ...,
          [ 0.3457, -0.0931, -0.0309,  ...,  0.6108,  0.2350,  0.2260],
          [ 0.0655,  0.4626,  0.2725,  ..., -0.1536,  0.1635, -0.0526],
          [ 0.0756,  0.1302,  0.1182,  ..., -0.0341,  0.2798, -0.0627]]],
        grad_fn=<ViewBackward>)}

In [35]:
q['code_perplexity']

tensor(4.9826)

In [47]:
model.embed

512

In [44]:
vq = GumbelVectorQuantizer(
    dim=model.embed,
    num_vars=model.cfg.latent_vars,
    temp=model.cfg.latent_temp,
    groups=model.cfg.latent_groups,
    combine_groups=False,
    vq_dim=model.cfg.final_dim,
    time_first=True,
)

In [55]:
r = vq(f)
r

{'num_vars': 640,
 'code_perplexity': tensor(11.0501),
 'prob_perplexity': tensor(601.1962, grad_fn=<SumBackward0>),
 'temp': 2.0,
 'x': tensor([[[0.9666, 0.6725, 0.7767,  ..., 0.5203, 0.1307, 0.3027],
          [0.6809, 0.7242, 0.8165,  ..., 0.9558, 0.7774, 0.1392],
          [0.4983, 0.8441, 0.0969,  ..., 0.2526, 0.4368, 0.8284],
          ...,
          [0.2042, 0.5790, 0.6596,  ..., 0.2189, 0.0719, 0.0449],
          [0.9599, 0.7449, 0.5854,  ..., 0.6189, 0.2629, 0.4825],
          [0.9998, 0.3339, 0.6949,  ..., 0.5752, 0.3326, 0.9932]],
 
         [[0.5799, 0.1077, 0.0963,  ..., 0.7342, 0.8199, 0.0784],
          [0.6574, 0.2049, 0.5405,  ..., 0.9150, 0.6552, 0.6104],
          [0.9243, 0.5255, 0.1761,  ..., 0.2053, 0.4012, 0.9608],
          ...,
          [0.4831, 0.0083, 0.5897,  ..., 0.7978, 0.1023, 0.9766],
          [0.6703, 0.3706, 0.1808,  ..., 0.1149, 0.5444, 0.4590],
          [0.0100, 0.4877, 0.6562,  ..., 0.5031, 0.9376, 0.3419]]],
        grad_fn=<ViewBackward>)}

In [49]:
model.cfg.latent_vars

320

In [43]:
model.cfg.latent_dim, model.cfg.final_dim, model.cfg.encoder_embed_dim

(0, 256, 768)

In [48]:
q['x'].shape

torch.Size([1, 31, 256])

In [50]:
model.cfg.latent_groups

2

In [56]:
(r['num_vars'] - r['prob_perplexity']) / r['num_vars']

tensor(0.0606, grad_fn=<DivBackward0>)